<a href="https://colab.research.google.com/github/PaintDumpster/Research_studio_code/blob/main/RATZCodeforNEO4J.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install neo4j pandas tqdm

In [23]:
import json
from neo4j import GraphDatabase
from tqdm import tqdm

class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response


In [39]:
def populate_neo4j(conn, data):
    with conn.__driver.session() as session:  # Use a transaction
        for wall_data in tqdm(data["walls"], total=len(data["walls"])):
            wall_id = wall_data["id"]
            materials = wall_data["materials"]

            # Create Wall node
            session.run("""
                MERGE (w:Wall {id: $wall_id})
            """, parameters={'wall_id': wall_id})

            # Create Material nodes and relationships
            for material in materials:
                session.run("""
                    MERGE (m:Material {name: $name})
                    ON CREATE SET m.density = $density, m.conductivity = $conductivity,
                                 m.u_value = $u_value, m.embodied_carbon_coefficient = $carbon_coeff,
                                 m.cost = $cost, m.recyclability = $recyclability,
                                 m.bio_based = $bio_based, m.color = $color
                    ON MATCH SET m.density = $density, m.conductivity = $conductivity,
                                 m.u_value = $u_value, m.embodied_carbon_coefficient = $carbon_coeff,
                                 m.cost = $cost, m.recyclability = $recyclability,
                                 m.bio_based = $bio_based, m.color = $color
                    MERGE (w:Wall {id: $wall_id})
                    MERGE (m)-[:USED_IN {thickness: $thickness}]->(w)
                """, parameters={
                    'name': material['material'],
                    'wall_id': wall_id,
                    'thickness': material['thickness'],
                    'density': material['density'],
                    'conductivity': material['conductivity'],
                    'u_value': material['u_value'],
                    'carbon_coeff': material['embodied_carbon_coefficient'],
                    'cost': material['cost'],
                    'recyclability': material['recyclability'],
                    'bio_based': material['bio_based'],
                    'color': material['color']
                })

In [25]:
if __name__ == "__main__":
    # Path to your JSON file
    file_path = '/content/wall_population.json'

    # Load JSON data
    print("Loading data...")
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Neo4j connection details
    uri = "neo4j+s://abc70c77.databases.neo4j.io"  # Replace with your Neo4j URI
    user = "neo4j"                 # Replace with your Neo4j username
    password = "cFpznjeEWz2vXfs2r2hDM93V2xNgGA1dDeQuV24-x4A"          # Replace with your Neo4j password

    # Establish connection
    print("Connecting to Neo4j...")
    conn = Neo4jConnection(uri, user, password)

    # Populate Neo4j database
    print("Populating the Neo4j database...")
    populate_neo4j(conn, data)

    # Close connection
    conn.close()
    print("Neo4j database population complete.")


Loading data...
Connecting to Neo4j...
Populating the Neo4j database...
Neo4j database population complete.
